### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import os
import csv
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.shape

(780, 7)

In [3]:
purchase_data.describe()

,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,91.755128,3.050987
std,225.310896,6.659444,52.697702,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,47.750000,1.980000
50%,389.500000,22.000000,92.000000,3.150000
75%,584.250000,25.000000,138.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [4]:
#checking for non-null cells
purchase_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Purchase ID  780 non-null    int64  
 1   SN           780 non-null    object 
 2   Age          780 non-null    int64  
 3   Gender       780 non-null    object 
 4   Item ID      780 non-null    int64  
 5   Item Name    780 non-null    object 
 6   Price        780 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.8+ KB


In [5]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [6]:
# sn_count = len(pd.unique(purchase_data['SN']))
sn_all = purchase_data['SN']
sn_unique = pd.unique(sn_all)
sn_count = len(sn_unique)
sn_count

576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [7]:
unique_items = len(pd.unique(purchase_data['Item Name']))
avg_price = round(purchase_data['Price'].mean(), 2)
num_purchase = len(purchase_data['Item Name'])
total_revenue = round(purchase_data['Price'].sum(), 2)

# change to currency
# apply(lambda x: "${:.1f}k".format((x/1000)))

purchasing_df = pd.DataFrame({
    "Number of Unique Items": [unique_items],
    "Average Price": [avg_price],
    "Number of Purchases": [num_purchase],
    "Total Revenue": [total_revenue]
})
purchasing_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.05,780,2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
#Group Gender by Unique SN
grouped_gen = purchase_data.groupby(['Gender'])['SN'].nunique()

#Total Unique SN Count
total_gender = grouped_gen.sum()

#Ratio of Male, Female, and Other based on Unique SN
male_ratio = round((grouped_gen['Male']/total_gender)*100, 2)
female_ratio = round((grouped_gen['Female']/total_gender)*100, 2)
other_ratio = round((grouped_gen['Other / Non-Disclosed']/total_gender)*100, 2)

#Create Dataframe
grouped_gen = grouped_gen.to_frame()

#Rename Columns
grouped_gen = grouped_gen.rename(index=str, columns={'Gender':'', 'SN': 'Total Count'})

#Add Column for Ratio
grouped_gen['Percentage of Players'] = [male_ratio, female_ratio, other_ratio]

#Print
grouped_gen




,Total Count,Percentage of Players
Gender,,
Female,81,84.03
Male,484,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
#Groupby Gender
purchase_gen = purchase_data.groupby(['Gender'])

# Purchase Count
purchase_gen_count = purchase_gen.count()['Purchase ID']
# purchase_gen_count
# print(purchase_gen_count)

# Average Purchase Price
purchase_gen_mean = round(purchase_gen.mean()['Price'], 2)

# Total Purchase Value
purchase_gen_sum = round(purchase_gen.sum()['Price'], 2)

# Avg Total Purchase per Person
grouped_gen = purchase_data.groupby(['Gender'])['SN'].nunique()
purchase_gen_pp = round(purchase_gen_sum / grouped_gen, 2)

# Create Dictionary
purchase_gen = {"Purchase Count": purchase_gen_count,
        "Average Purchase Price": purchase_gen_mean,
        "Total Purchase Value": purchase_gen_sum,
        "Avg Total Purchase per Person": purchase_gen_pp}

# Concatenate Multiple Series and Create DataFrame
purchase_gen_df = pd.concat(purchase_gen,
               axis = 1)

#Print
purchase_gen_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
# Min age is 7; Max age is 45; Mean age is 22.

# Create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
bin_labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Put players into Age bins
purchase_age = purchase_data
purchase_age['Age Bins'] = pd.cut(purchase_age['Age'], bins, labels = bin_labels, include_lowest = True)

# For number of unique SNs groupby Age bins
purchase_age_sn = purchase_age.groupby('Age Bins').apply(lambda x: len(x['SN'].unique()))

# Create dataframe
purchase_age_sn = pd.DataFrame(purchase_age_sn)

# Create "Total Count" and "Percentage of Players" columns and perform calculations
purchase_age_sn.columns = ['Total Count']
purchase_age_sn['Percentage of Players'] = round(purchase_age_sn['Total Count'] / sn_count * 100, 2)

# Call dataframe
purchase_age_sn

,Total Count,Percentage of Players
Age Bins,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
# Create bins
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
bin_labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

# Put players into Age bins
purchase_age = purchase_data
purchase_age['Age Bins'] = pd.cut(purchase_age['Age'], bins, labels = bin_labels, include_lowest = True)

#Unique SNs by age
age_sn = purchase_age.groupby('Age Bins').apply(lambda x: len(x['SN'].unique()))

#Average Purchase Price and Total Purchase Value 
purchase_analysis = round(purchase_age.groupby('Age Bins').agg({'Price': ['mean', 'sum']}), 2)
purchase_analysis2 = purchase_age.groupby('Age Bins').apply(lambda x: len(x['SN']))


# Create df
purchase_analysis2 = pd.DataFrame(purchase_analysis2)
purchase_analysis = pd.merge(purchase_analysis, purchase_analysis2, on = 'Age Bins')

# Rename Columns
purchase_analysis.columns = ['Average Purchase Price', 'Total Purchase Value', 'Purchase Count']

# Average Total Purchase per person
purchase_analysis['Average Total Purchase per person'] = round((purchase_analysis['Total Purchase Value'] / age_sn),2)

# Update df
purchase_analysis['Average Purchase Price'] = purchase_analysis['Average Purchase Price']
purchase_analysis['Total Purchase Value'] = purchase_analysis['Total Purchase Value']
purchase_analysis['Average Total Purchase per person'] = purchase_analysis['Average Total Purchase per person']

# Print
purchase_analysis

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,Average Purchase Price,Total Purchase Value,Purchase Count,Average Total Purchase per person
Age Bins,,,,
<10,3.35,77.13,23,4.54
10-14,2.96,82.78,28,3.76
15-19,3.04,412.89,136,3.86
20-24,3.05,1114.06,365,4.32
25-29,2.90,293.00,101,3.81
30-34,2.93,214.00,73,4.12
35-39,3.60,147.67,41,4.76
40+,2.94,38.24,13,3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [12]:
#Groupby SN
purchase_sn = purchase_data.groupby(['SN'])

# Purchase Count
purchase_sn_count = purchase_sn.count()['Purchase ID']

# Average Purchase Price
purchase_sn_mean = round(purchase_sn.mean()['Price'], 2)

# Total Purchase Value
purchase_sn_sum = round(purchase_sn.sum()['Price'], 2)

# Create Dictionary
purchase_sn = {"Purchase Count": purchase_sn_count,
        "Average Purchase Price": purchase_sn_mean,
        "Total Purchase Value": purchase_sn_sum}

# Concatenate Multiple Series and Create DataFrame
purchase_sn_df = pd.concat(purchase_sn,
               axis = 1)

# Sort in Descending Order
purchase_sn_df.sort_values(by=['Total Purchase Value'], inplace=True, ascending=False)

print(type(purchase_sn_count))


#Print
purchase_sn_df.head()

<class 'pandas.core.series.Series'>


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [15]:
# Purchase Count
purchase_id_count = purchase_data.groupby("Item ID").count()

# purchase_id_count = purchase_data.groupby(['Item ID', 'Item Name']).count()
# purchase_id_count['Purchase ID']

purchase_id_count = purchase_id_count['Purchase ID']

# Item Name
purchase_id_name = purchase_data['Item Name']

# Total Purchase Value
purchase_id_price = purchase_data['Price']
purchase_id_total = round(purchase_id_count * purchase_id_price, 2)


# Create Dictionary
purchase_id_dic = {"Item ID": purchase_data['Item ID'],
               "Item Name": purchase_data['Item Name'],
               "Purchase Count": purchase_id_count,
               "Item Price": purchase_id_price,
               "Total Purchase Value": purchase_id_total 
              }

# Concatenate Multiple Series and Create DataFrame
purchase_id_df = pd.concat(purchase_id_dic,
               axis = 1)

#Print
purchase_id_df.head()


,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
0,108,"Extraction, Quickblade Of Trembling Hands",4.0,3.53,14.12
1,143,Frenzied Scimitar,4.0,1.56,6.24
2,92,Final Critic,6.0,4.88,29.28
3,100,Blindscythe,6.0,3.27,19.62
4,131,Fury,5.0,1.44,7.20


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [14]:
# Sort in Descending Order
purchase_id_df.sort_values(by=["Total Purchase Value"], inplace=True, ascending=False)
purchase_id_df.head()

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
178,57,"Despair, Favor of Due Diligence",12.0,4.60,55.20
92,39,"Betrayal, Whisper of Grieving Widows",13.0,3.94,51.22
132,145,Fiery Glass Crusader,9.0,4.58,41.22
82,160,Azurewrath,9.0,4.40,39.60
19,89,"Blazefury, Protector of Delusions",8.0,4.64,37.12
